In [1]:
from bs4 import BeautifulSoup as bs
from bs4.element import Comment
import pandas as pd
import numpy as np
import requests

#Data Question 4: Web Scraping Earthquake Data
##Part 1:

Nate Silver discusses the difficulty of predicting earthquakes in The Signal and the Noise. Nevertheless, we will try to identify some patterns by analyzing the deadly earthquakes that have occurred since 1900.

To start, read the table of earthquakes from https://en.wikipedia.org/wiki/List_of_deadly_earthquakes_since_1900 using the requests and/or beautifulsoup library and load it to a pandas dataframe. You will need to do some data cleaning before you can proceed.

In [2]:
url='https://en.wikipedia.org/wiki/List_of_deadly_earthquakes_since_1900'

In [8]:
earthq_resp=requests.get(url).content

In [ ]:
print(earthq_resp.status_code)

In [13]:
# parse data from the html into a beautifulsoup object
earthq_soup=bs(earthq_resp, 'html.parser')
earthq_table=earthq_soup.find('table',{'class':"wikitable"})

In [ ]:
earthq_table 

In [ ]:
#convert html to list of data frame object
earthq_df=pd.read_html(str(earthq_table))
# convert list to dataframe
earthq_df=pd.DataFrame(earthq_df[0])
print(earthq_df.head())

##Data cleaning tasks include:

- Replace empty strings with NaN
- Remove the footnotes from the 'Other Source Deaths' column
- Convert Magnitude to a numeric type. For this portion, you can ignore differences in seismic magnitude scales.
- Correct number of deaths when there is more than one value. When there is more than one value given, choose the largest.
- Create a new column ('deaths') that evaluates the four total-death columns ('PDE Total Deaths', 'Utsu Total Deaths', 'EM-DAT Total Deaths', and 'Other Source Deaths') and populates the new column with the highest value.
- Explore the data in terms of when and where earthquakes occurred and how severe they were (magnitude, deaths, secondary effects).

In [18]:
earthq_df=earthq_df.replace(r'^\s*$', np.NaN, regex=True)

In [ ]:
earthq_df. head(60)  

In [ ]:
earthq_df. tail(60)

In [60]:
earthq_df['OtherSourceDeathsClean']=earthq_df['Other Source Deaths'].str.extract(r'(\d+)\D*').astype('float')

In [57]:
earthq_df['Magnitude_num']=earthq_df['Magnitude'].str.extract(r'(\d+\.\d+)\w*\D*\d*\D*').astype('float')

In [58]:
earthq_df['Magnitude_type']=earthq_df['Magnitude'].str.extract(r'\d+\.\d+\s*(\w+)')

In [72]:
earthq_df.head(60)

,Origin (UTC),Present-day country and link to Wikipedia article,Lat,Long,Depth (km),Magnitude,Secondary Effects,PDE Shaking Deaths,PDE Total Deaths,Utsu Total Deaths,EM-DAT Total Deaths,Other Source Deaths,OtherSourceDeathsClean,Magnitude_num,Magnitude_type,max_deaths,EM-DAT_TotalDeathsClean
0,1900-05-11 17:23,Japan,38.700,141.100,5,7.0 MJMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,MJMA,NaN,NaN
1,1900-07-12 06:25,Turkey,40.300,43.100,NaN,5.9 Muk,NaN,NaN,NaN,140.0,NaN,NaN,NaN,5.9,Muk,140.0,NaN
2,1900-10-29 09:11,Venezuela,11.000,-66.000,0,7.7 Mw,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.7,Mw,NaN,NaN
3,1901-02-15 00:00,China,26.000,100.100,0,6.5 Ms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.5,Ms,NaN,NaN
4,1901-03-31 07:11,Bulgaria,43.400,28.700,NaN,6.4 Muk,NaN,NaN,NaN,4.0,NaN,NaN,NaN,6.4,Muk,4.0,NaN
5,1901-08-09 09:23,Japan,40.500,142.500,35,7.2 Mw,T,NaN,NaN,NaN,NaN,NaN,NaN,7.2,Mw,NaN,NaN
6,1901-11-15 20:15,New Zealand (see 1901 Cheviot earthquake),-43.000,173.000,0,6.8 Ms,NaN,NaN,NaN,1.0,NaN,NaN,NaN,6.8,Ms,1.0,NaN
7,1902-01-30 14:01,Japan,40.500,141.300,35,6.9 Ms,NaN,NaN,NaN,1.0,NaN,NaN,NaN,6.9,Ms,1.0,NaN
8,1902-02-13 09:39,Azerbaijan,40.700,48.600,15,6.9 Muk,NaN,NaN,NaN,86.0,NaN,NaN,NaN,6.9,Muk,86.0,NaN
9,1902-03-09 07:46,Turkey,40.700,33.600,NaN,5.5 Muk,NaN,NaN,NaN,4.0,NaN,NaN,NaN,5.5,Muk,4.0,NaN


In [70]:
earthq_df.tail(60)

,Origin (UTC),Present-day country and link to Wikipedia article,Lat,Long,Depth (km),Magnitude,Secondary Effects,PDE Shaking Deaths,PDE Total Deaths,Utsu Total Deaths,EM-DAT Total Deaths,Other Source Deaths,OtherSourceDeathsClean,Magnitude_num,Magnitude_type,max_deaths,EM-DAT_TotalDeathsClean
1280,2007-01-21 11:27,Indonesia,1.065,126.282,22,7.5 Mw,NaN,3.0,4.0,NaN,NaN,NaN,NaN,7.5,Mw,4.0,NaN
1281,2007-03-06 03:49,Indonesia (see March 2007 Sumatra earthquakes),-0.480,100.470,19.3,6.4 Mw,NaN,67.0,67.0,NaN,67,NaN,NaN,6.4,Mw,67.0,67.0
1282,2007-03-25 00:41,Japan (see 2007 Noto earthquake),37.340,136.540,8,6.7 Mw,T,1.0,1.0,NaN,1,NaN,NaN,6.7,Mw,1.0,1.0
1283,2007-04-01 20:39,Solomon Islands (see 2007 Solomon Islands eart...,-8.430,157.060,10,8.1 Mw,"T,L",0.0,54.0,NaN,NaN,NaN,NaN,8.1,Mw,54.0,NaN
1284,2007-04-21 17:53,Chile (see 2007 Aysén Fjord earthquake),-45.240,-72.670,36.7,6.2 Mw,"T,L",0.0,10.0,NaN,10,NaN,NaN,6.2,Mw,10.0,10.0
1285,2007-06-02 21:34,China,23.020,101.010,5,6.1 Mw,L,3.0,3.0,NaN,3,NaN,NaN,6.1,Mw,3.0,3.0
1286,2007-07-16 01:13,Japan (see 2007 Chūetsu offshore earthquake),37.520,138.460,12,6.6 Mw,"T,L",9.0,9.0,NaN,9,NaN,NaN,6.6,Mw,9.0,9.0
1287,2007-07-21 22:44,Tajikistan,38.936,70.485,10,5.2 Mw,L,3.0,12.0,NaN,11,NaN,NaN,5.2,Mw,12.0,11.0
1288,2007-08-02 02:37,Russian Federation,47.110,141.810,5,6.2 Mw,T,2.0,2.0,NaN,2,NaN,NaN,6.2,Mw,2.0,2.0
1289,2007-08-15 23:40,Peru (see 2007 Peru earthquake),-13.380,-76.610,39,8.0 Mw,L,514.0,514.0,NaN,519,NaN,NaN,8.0,Mw,514.0,519.0


In [69]:
earthq_df['EM-DAT_TotalDeathsClean']= earthq_df['EM-DAT Total Deaths'].str.extract(r'(\d+)\s*\D*').astype('float')


In [71]:
earthq_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 17 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   Origin (UTC)                                       1340 non-null   object 
 1   Present-day country and link to Wikipedia article  1340 non-null   object 
 2   Lat                                                1325 non-null   object 
 3   Long                                               1325 non-null   object 
 4   Depth (km)                                         1250 non-null   object 
 5   Magnitude                                          1339 non-null   object 
 6   Secondary Effects                                  373 non-null    object 
 7   PDE Shaking Deaths                                 738 non-null    float64
 8   PDE Total Deaths                                   749 non-null    float64
 9   Utsu Tot

In [75]:
earthq_df['max_deaths'] = earthq_df[[
    'PDE Shaking Deaths','PDE Total Deaths','Utsu Total Deaths','EM-DAT_TotalDeathsClean', 'OtherSourceDeathsClean'
]].max(axis=1)

In [79]:
earthq_df.tail(60)

,Origin (UTC),Present-day country and link to Wikipedia article,Lat,Long,Depth (km),Magnitude,Secondary Effects,PDE Shaking Deaths,PDE Total Deaths,Utsu Total Deaths,EM-DAT Total Deaths,Other Source Deaths,OtherSourceDeathsClean,Magnitude_num,Magnitude_type,max_deaths,EM-DAT_TotalDeathsClean
1280,2007-01-21 11:27,Indonesia,1.065,126.282,22,7.5 Mw,NaN,3.0,4.0,NaN,NaN,NaN,NaN,7.5,Mw,4.0,NaN
1281,2007-03-06 03:49,Indonesia (see March 2007 Sumatra earthquakes),-0.480,100.470,19.3,6.4 Mw,NaN,67.0,67.0,NaN,67,NaN,NaN,6.4,Mw,67.0,67.0
1282,2007-03-25 00:41,Japan (see 2007 Noto earthquake),37.340,136.540,8,6.7 Mw,T,1.0,1.0,NaN,1,NaN,NaN,6.7,Mw,1.0,1.0
1283,2007-04-01 20:39,Solomon Islands (see 2007 Solomon Islands eart...,-8.430,157.060,10,8.1 Mw,"T,L",0.0,54.0,NaN,NaN,NaN,NaN,8.1,Mw,54.0,NaN
1284,2007-04-21 17:53,Chile (see 2007 Aysén Fjord earthquake),-45.240,-72.670,36.7,6.2 Mw,"T,L",0.0,10.0,NaN,10,NaN,NaN,6.2,Mw,10.0,10.0
1285,2007-06-02 21:34,China,23.020,101.010,5,6.1 Mw,L,3.0,3.0,NaN,3,NaN,NaN,6.1,Mw,3.0,3.0
1286,2007-07-16 01:13,Japan (see 2007 Chūetsu offshore earthquake),37.520,138.460,12,6.6 Mw,"T,L",9.0,9.0,NaN,9,NaN,NaN,6.6,Mw,9.0,9.0
1287,2007-07-21 22:44,Tajikistan,38.936,70.485,10,5.2 Mw,L,3.0,12.0,NaN,11,NaN,NaN,5.2,Mw,12.0,11.0
1288,2007-08-02 02:37,Russian Federation,47.110,141.810,5,6.2 Mw,T,2.0,2.0,NaN,2,NaN,NaN,6.2,Mw,2.0,2.0
1289,2007-08-15 23:40,Peru (see 2007 Peru earthquake),-13.380,-76.610,39,8.0 Mw,L,514.0,514.0,NaN,519,NaN,NaN,8.0,Mw,519.0,519.0
